# SK GitHub MCP Server Demo

## Setup

### Load required .NET packages and supporting constants, classes, etc.

In [ ]:
#r "nuget: dotenv.net"
#r "nuget: Microsoft.SemanticKernel, 1.49.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.sAzureOpenAI, 1.49.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.49.0"
#r "nuget: ModelContextProtocol, 0.2.0-preview.1"
#r "nuget: dotenv.net"

using System;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;
using Microsoft.SemanticKernel.Agents;
using ModelContextProtocol.Client;

using dotenv.net;


#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003
#pragma warning disable SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025
#pragma warning disable SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,
#pragma warning disable SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102

Installed Packages dotenv.net, 3.2.1 Microsoft.SemanticKernel, 1.49.0 Microsoft.SemanticKernel.Agents.Core, 1.49.0 Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.49.0 ModelContextProtocol, 0.2.0-preview.1

### Start the Github MCP Server

In [2]:
var projectPath = @"/home/alex/github/msalemor/sk-dev-training/notebooks/mcp/mcp1/server";

var mcpClient = await McpClientFactory.CreateAsync(
    new StdioClientTransport(new () {
        Name = "GitHub",
        Command = "npx",
        Arguments = ["@modelcontextprotocol/server-github"],
    })
);

### List all available tools in the MCP server

In [3]:
var tools = await mcpClient.ListToolsAsync().ConfigureAwait(false);
foreach (var tool in tools)
{
    Console.WriteLine($"{tool.Name}: {tool.Description}");
}

create_or_update_file: Create or update a single file in a GitHub repository
search_repositories: Search for GitHub repositories
create_repository: Create a new GitHub repository in your account
get_file_contents: Get the contents of a file or directory from a GitHub repository
push_files: Push multiple files to a GitHub repository in a single commit
create_issue: Create a new issue in a GitHub repository
create_pull_request: Create a new pull request in a GitHub repository
fork_repository: Fork a GitHub repository to your account or specified organization
create_branch: Create a new branch in a GitHub repository
list_commits: Get list of commits of a branch in a GitHub repository
list_issues: List issues in a GitHub repository with filtering options
update_issue: Update an existing issue in a GitHub repository
add_issue_comment: Add a comment to an existing issue
search_code: Search for code across GitHub repositories
search_issues: Search for issues and pull requests across GitHub re

### Read the API Key and endpoints from environment variables or the .env file

In [4]:
// Load the .env file
DotEnv.Load();

// Get the OpenAI deployment name, endpoint, and key from the environment variables
var deploymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");

### Configure the Kernel and memory

In [5]:
// Note: Added this because I am having problems with SSL certificate validation
var handler = new HttpClientHandler();
handler.CheckCertificateRevocationList = false;
var httpClient = new HttpClient(handler);

In [6]:
#pragma warning disable SKEXP0010
var kernelBuilder = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: deploymentName,
        endpoint: endpoint,
        apiKey: apiKey,
        httpClient: httpClient);
Kernel kernel = kernelBuilder.Build();

### Add the MCP server as an SK plugin and use it automatically

In [7]:
#pragma warning disable SKEXP0001
kernel.Plugins.AddFromFunctions("GitHub", tools.Select(aiFunction => aiFunction.AsKernelFunction()));

#pragma warning disable SKEXP0001
AzureOpenAIPromptExecutionSettings executionSettings = new()
{
    Temperature = 0,
    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto(options: new() { RetainArgumentTypes = true })
};

### Test the MCP server in a prompt

In [8]:
// Test using GitHub tools
var prompt = "Summarize the last four commits to the microsoft/semantic-kernel repository?";
var result = await kernel.InvokePromptAsync(prompt, new(executionSettings)).ConfigureAwait(false);
Console.WriteLine($"\n\n{prompt}\n{result}");



Summarize the last four commits to the microsoft/semantic-kernel repository?
Here are the summaries of the last four commits to the `microsoft/semantic-kernel` repository:

1. **Commit SHA:** [5e488d2](https://github.com/microsoft/semantic-kernel/commit/5e488d2b7f6935e2b81b14c81cd2680608c899ed)  
   **Author:** westey  
   **Date:** 2025-05-22  
   **Message:** Bumped the NuGet package version to 1.54.0. This update ensures the repository aligns with the latest versioning standards.  

2. **Commit SHA:** [cecee4c](https://github.com/microsoft/semantic-kernel/commit/cecee4cb6a191a1f1833e9248fdf7738bf2556a5)  
   **Author:** Roger Barreto  
   **Date:** 2025-05-22  
   **Message:** Fixed MCP behavior for `AIFunction.AsKernelFunction()` and ensured `FunctionChoiceBehavior` works correctly with `IChatClients`. This commit addresses issues #12209 and #12217.

3. **Commit SHA:** [912a7bb](https://github.com/microsoft/semantic-kernel/commit/912a7bb0222e34ab703209c5402c9ff52e96761d)  
   **A